## Environment

In [25]:
import numpy as np
import time
import math
import random
import matplotlib.pyplot as plt 
from scipy.optimize import minimize, Bounds, linprog
# Environment parameters
K = 100  # Number of content items
u_min = 0.8  # Threshold for content relevance
C = int(0.8 * K)  # Number of cached items
#C = 2
# User model parameters
N = 4 # Number of recommended items
q = 0.2  # Probability of ending the viewing session
alpha = 0.8  # Probability of selecting a recommended item
#tradeoff_factor=0.4
# Generate random relevance values
U = np.random.rand(K, K)
np.fill_diagonal(U, 0)  # Set diagonal elements to 0
# U = np.array([[0., 0.09709217, 0.95697935, 0.76421269, 0.79379138],
#               [0.85679266, 0., 0.73115609, 0.97025111, 0.00706508],
#               [0.38327773, 0.27582305, 0., 0.40938946, 0.70918518],
#               [0.27415892, 0.89691232, 0.47103534, 0., 0.97776446],
#               [0.06699551, 0.96500574, 0.00547615, 0.74654658, 0.]])
# U = np.array([[0.0, 0.8, 0.3, 0.6],
#               [0.8, 0.0, 0.7, 0.2],
#               [0.3, 0.1, 0.0, 0.2],
#               [0.6, 0.4, 0.2, 0.0]])
print(U)
#vector to denote the cost of each state. 1 for non-cached, 0 for cached
Cost = [1]*(K-C) +[0]*C   
#Cost = [1,0,1,0]
random.shuffle(Cost)
print(Cost)

Tmax = 10000

#create action set as the set of every possible combination of N=2 states 
# action_set = []
# for i in range(K):
#     for j in range(i+1,K):
#         a = (i, j)
#         action_set.append(a)

# num_of_actions = len(action_set)

# action_table = [[] for _ in range(K)]

# for i in range(K):
#     for a in action_set:
#         if i not in a:
#             action_table[i].append(a)
# num_of_actions = len(action_table[0])
# print(num_of_actions)
# print (action_table)
# a = random.choice(action_table[2])
# s_prime = int(np.random.choice(a))
# print(a)
# print(s_prime)

def normalize_matrix_R(R):
    row_sums = R.sum(axis=1)

    # New matrix is old matrix divided by row sums
    # Use np.newaxis to match the dimensions for broadcasting
    R = N*R / row_sums[:, np.newaxis]
    return R

# R = normalize_matrix_R(U)
# print(R.sum(axis=1))

[[0.         0.11191734 0.79896733 ... 0.68292715 0.03872294 0.18091375]
 [0.52668159 0.         0.8884091  ... 0.71369419 0.39868969 0.52831082]
 [0.92881528 0.24741652 0.         ... 0.61819665 0.34488573 0.46754078]
 ...
 [0.55356429 0.38491872 0.34322512 ... 0.         0.04005022 0.56609825]
 [0.44808562 0.24297506 0.27416776 ... 0.24672608 0.         0.05742574]
 [0.91890314 0.99584594 0.83795267 ... 0.97545181 0.14907142 0.        ]]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0]


## Approximate Policy Iteration


In [26]:



def all_recommendations_are_relevant(recommendations,s):
    """
    function to check whether everey recommended state in the racommendation batch is 
    relevant to s

    arguments:
    recommendations (tuple of ints): recommendation batch for state s
    s (int): current state

    returns:
    True: if all recommendation are relevant to s
    False: otherwise
    """
    for u in recommendations:
        if U[s][int(u)]<u_min:
            return False
    return True

# def get_next_states(s, r): 
#     """
#     function to generate all possible next states alongiside their respective probabilities and rewards
#     given a state s and an action a

#     arguments:
#     s (int):  current state
#     a (tuple of ints): recommendation batch for state s

#     returns:
#     next_states (list of tuples):a list of tuples, where each tuple denotes the next state alongiside its respective probability and reward
#     """
#     next_states = []
    
#     for s_prime in range(K):
        
#         prob = (1-N*u_min)*(alpha * r[s_prime]/N + (1-alpha)/K) +N/K*u_min #probability to choose a video from the recommedation batch 
#         reward = 1/2-Cost[s_prime] #define reward to be 1 for cached and 0 for uncached video
#         next_states.append((prob,s_prime,reward))
        
                
#     return next_states
       
# def policy_evaluation(R, gamma = 1.0, epsilon = 1e-10):  
#     """
#     Function to evaluate a given policy

#     arguments:
#     pi (list of tuples): policy to be evaluated
#     gamma (float): discounting factor
#     epsilon (float): approximation tolerance

#     returns:
#     V (vector of floats): vector of value function of each state calculated using bellman equation
#     """

#     prev_V = np.zeros(K) # use as "cost-to-go", i.e. for V(s')
#     while True: #performing iterations
#         V = np.zeros(K) # current value function to be learnerd
#         for s in range(K):  # do for every state
#             for prob, next_state, reward in get_next_states(s, R[s]):  # calculate one Bellman step --> i.e., sum over all probabilities of transitions and reward for that state, the action suggested by the (fixed) policy, the reward earned (dictated by the model), and the cost-to-go from the next state (which is also decided by the model)
#                 V[s] += prob * (reward + gamma * prev_V[next_state] )
#         if np.max(np.abs(prev_V - V)) < epsilon: #check if the new V estimate is close enough to the previous one; 
#             break # if yes, finish loop
#         prev_V = V.copy() #freeze the new values (to be used as the next V(s'))
#     return V



# # Define the function to minimize
# def func(x,V):
#         return -np.dot(x , V )

# def policy_improvement(R, V, gamma=1.0):  
#     """
#     Function to greedily improve policy

#     arguments:
#     V (vector of floats): V (vector of floats): vector of value function of each state calculated using bellman equation
#     gamma (float): discounting factor
    
#     returns:
#     new_pi (list of tuples): new policy
#     """
   
#     # Define the equality constraint
    
#     new_R = np.zeros((K, K), dtype=np.float64) #create a Q value array
#     # # Print the result
#     # print(f"Minimized function value: {result.fun}")
#     # print(f"Argmin: {result.x}")

#     for s in range(K):
        
#         constraints = [{'type': 'eq', 'fun': lambda x: sum(x) - N},  # Sum of elements should be N
#                         {'type': 'eq', 'fun': lambda x: x[s]}]
#         # Define the bounds (0 <= v(i) <= 1 for all i)
#         bounds = Bounds([0]*K, [1]*K)

#         # Initial guess
#         x0 = R[s]  # Initial guess should respect the constraints

#         # Call the minimize() function with the 'SLSQP' method, bounds and constraints
#         result = minimize(func, args = (V), x0=x0, method='SLSQP', bounds=bounds, constraints=constraints)
#         new_R[s] = result.x
#     return new_R


# def policy_iteration( gamma = 1.0, epsilon = 1e-10):
#     """
#     Function to converge to the optimal policy by performing iterative avaluation and improvments until convergence

#     arguments:
#     gamma (float): discounting factor
#     epsilon (float): approximation tolerance
    
#     returns:
#     pi (list of tuples): optimal policy
#     V (vector of floats): vector of value function of each state calculated using bellman equation
#     """
#     t = 0
#     # random_actions = np.random.choice(tuple(P[0].keys()), len(P))     # start with random actions for each state  
#     # pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]     # and define your initial policy pi_0 based on these action (remember, we are passing policies around as python "functions", hence the need for this second line)
#     R = normalize_matrix_R(U)
#     pi = np.random.randint(0, K, size=(K, N))  # Random initial policy
#     #print(np.sum(R,axis=1))

#     while True:
#         prev_R = R.copy()  #keep the old policy to compare with new
#         V = policy_evaluation(R,gamma,epsilon)   #evaluate latest policy --> you receive its converged value function

#         R = policy_improvement(R,V,gamma)          #get a better policy using the value function of the previous one just calculated 
        
#         t += 1

#         if (np.max(np.abs(prev_R - R)) < epsilon): # you have converged to the optimal policy if the "improved" policy is exactly the same as in the previous step
#             break
#     print('converged after %d iterations' %t) #keep track of the number of (outer) iterations to converge
    
#     #Construct a new police by selecting N items withs max r_ij
#     pi = np.argpartition(R, -N, axis=1)[:, -N:]

#     # Construct the KxN matrix using the indices
    
#     return V,pi
    
# def compare_two_policies(pi,old_pi):
#     """
#     Function to compare if two policies are identical

#     arguments:
#     pi: current policy
#     old: previous policy

#     returns:
#     True if they are identical
#     Flase otherwise
#     """
#     for k in range(K):
#         for n in range(N):
#             if (pi[k][n]!=old_pi[k][n]):
#                 return False

#     return True


# V_opt,P_opt1 = policy_iteration(1-q,0.001)   #just example of calling the various new functions we created.



# print("#############################")
# #print(V_opt)
# print("#############################")
# print(P_opt1)

## Approximate Q Learning

In [27]:

def func(x,Q):
        return -np.dot(x , Q )/N

def maximize_model(Q,s):
    new_P = np.zeros((K, K), dtype=np.float64) #create a Q value array
    # # Print the result
    # print(f"Minimized function value: {result.fun}")
    # print(f"Argmin: {result.x}")

    
    C=-1/N*Q
    # Equality constraint for summation: sum(x) = N
    A_eq = [[1] * K]
    b_eq = [N]

    # Adding the specific constraint x_i = 0
    # You can directly specify this in bounds, no need to add in A_eq

    # Bounds for each variable: 0 <= x_j <= 1 and x_i = 0
    bounds = [(0, 1) if j != s else (0, 0) for j in range(K)]

    # Now use linprog
    result = linprog(C, bounds=bounds, A_eq=A_eq, b_eq=b_eq, method='highs')
    new_P = result.x
    max = -result.fun
    return new_P,max

def maximize_model2(P,Q,s):
    new_P = np.zeros((K, K), dtype=np.float64) #create a Q value array
    # # Print the result
    # print(f"Minimized function value: {result.fun}")
    # print(f"Argmin: {result.x}")

    
        
    constraints = [{'type': 'eq', 'fun': lambda x: sum(x) - N},  # Sum of elements should be N
                    {'type': 'eq', 'fun': lambda x: x[s]}]
    # Define the bounds (0 <= v(i) <= 1 for all i)
    bounds = Bounds([0]*K, [1]*K)

    # Initial guess
    x0 = P  # Initial guess should respect the constraints

    # Call the minimize() function with the 'SLSQP' method, bounds and constraints
    result = minimize(func, args = (Q), x0=x0, method='SLSQP', bounds=bounds, constraints=constraints)
    new_P = result.x
    max = -result.fun
    return new_P,max


def Q_learning(gamma, epsilon,learning_rate):
    """
    Function to perform Q learning algorithm.

    arguments:
    gamma (float): discounting factor
    epsilon (float): epslilon greedy probability
    learning_rate (float): learning rate

    returns:
    Q (matrix K x num_of_actions):  martix of state action value function calculated using bellman equation
    """
    Q = np.zeros((K,K)) 
    P = normalize_matrix_R(U) 
    #prev_Q = np.zeros((K,K))
    t = 0
    while True:
        s = np.random.randint(K) #random initial state
        while True:
            if np.random.uniform() < epsilon:  # Explore if e(t) times
                
                numbers = list(range(K))
                numbers.remove(s)

                # Sample two distinct integers from the list
                a = tuple(random.sample(numbers, N))
                #s_prime = np.random.randint(K) #choose random action
                    
            else:  # Exploit 1-e(t) times
                a= tuple(np.argpartition(-np.array(P[s]), N)[:N])
                #s_prime = np.argmax(Q[s]) #choose greedily the action with highest Q value
            # a = action_table[s][a_idx]  


            if (all_recommendations_are_relevant(a,s)):
            
                
                s_prime = int(np.random.choice(a))  # Pick a random item from relevant recommended items
                
            else:
                s_prime = np.random.randint(K)  # Pick a random item

            if np.random.uniform() < q: #if user opt to terminate session
                target = (1 - 2*Cost[s_prime])
                Q[s][s_prime] = Q[s][s_prime] + learning_rate * ( target - Q[s][s_prime] )
                break
            else:
                P[s_prime],max = maximize_model(Q[s_prime],s_prime)
                target = (1 - 2*Cost[s_prime]) + gamma*max
            Q[s][s_prime] = Q[s][s_prime] + learning_rate * ( target - Q[s][s_prime] )
            
            s = s_prime
        t+=1
        epsilon = (t+1)**(-1/3)*(K*math.log(t+1))**(1/3)
        #epsilon = 0.1
        #learning_rate = learning_rate*(1/t)**(1/2)
        
        #if (np.max(np.abs(prev_Q - Q)) < delta and t>1000*K) or 
        if t > 100*K: #check if the new V estimate is close enough to the previous one;
            break # if yes, finish loop
        #prev_Q = Q.copy()
    print(Q)
    print("===================================================================================")
    print(P)
    return P



# pi_Q_learning  =  np.zeros((K, N), dtype=np.int16)
print(U)
print(Cost)
print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
Q = Q_learning(1-q,1,0.01)

pi_Q_learning = np.argpartition(Q, -N, axis=1)[:, -N:]
# for s in range(K):
#     #Q[s][s] = float('-inf')
#     action = np.argmax(Q[s])
#     pi_Q_learning[s] = action_table[s][action]

print(pi_Q_learning)

[[0.         0.11191734 0.79896733 ... 0.68292715 0.03872294 0.18091375]
 [0.52668159 0.         0.8884091  ... 0.71369419 0.39868969 0.52831082]
 [0.92881528 0.24741652 0.         ... 0.61819665 0.34488573 0.46754078]
 ...
 [0.55356429 0.38491872 0.34322512 ... 0.         0.04005022 0.56609825]
 [0.44808562 0.24297506 0.27416776 ... 0.24672608 0.         0.05742574]
 [0.91890314 0.99584594 0.83795267 ... 0.97545181 0.14907142 0.        ]]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0]
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
[[ 0.05052386  0.02097428  0.03126905 ... -0.0744434   0.070451
   0.03091439]
 [ 0.02059317  0.05054273  0.02015761 ... -0.07358232  0.0614256
   0.        ]
 [ 0.0

## Simulation

In [28]:
def simulate_session(policy, max_steps=1000):
    """
    Simulate a viewing session following a given policy

    arguments:
    policy to be simulated

    returns:
    total cost of the session
    
    """
    s = np.random.randint(K)  # random initial
    cost_total = Cost[s]  
    for _ in range(max_steps):
        if np.random.uniform() < q:  # The user decides to quit
            break

        if (all_recommendations_are_relevant(policy[s],s)):
            
            if np.random.uniform() < alpha:  # If all recommended items are relevant
                s_prime = int(np.random.choice(policy[s]))  # Pick a random item from relevant recommended items
            else:  # If at least one recommended item is not relevant
                s_prime = np.random.randint(K)  # Pick a random item
        else:
            s_prime = np.random.randint(K)  # Pick a random item
        
        s=s_prime
        cost_total += Cost[s]  # Add the cost of the picked item
    return cost_total

def simulation(policy):
    """
    function to run multiple sessions
    """
    total_cost = 0
    num_of_episodes=50000
    for _ in range(num_of_episodes):
        total_cost  += simulate_session(policy)

    print(total_cost/num_of_episodes)

print(U)
print(Cost)
#print(P_opt1)
print(pi_Q_learning)

print("average cost for Policy iteration:")
#simulation(P_opt1)
#simulation(P_opt2)
print("average cost for Q Learning:")
simulation(pi_Q_learning)

[[0.         0.11191734 0.79896733 ... 0.68292715 0.03872294 0.18091375]
 [0.52668159 0.         0.8884091  ... 0.71369419 0.39868969 0.52831082]
 [0.92881528 0.24741652 0.         ... 0.61819665 0.34488573 0.46754078]
 ...
 [0.55356429 0.38491872 0.34322512 ... 0.         0.04005022 0.56609825]
 [0.44808562 0.24297506 0.27416776 ... 0.24672608 0.         0.05742574]
 [0.91890314 0.99584594 0.83795267 ... 0.97545181 0.14907142 0.        ]]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0]
[[90 22 69 72]
 [71  8 89 26]
 [26 35 85 87]
 [60 88 21 86]
 [96 63 68  1]
 [47  7 77 26]
 [82 34 43 86]
 [53 61 32 75]
 [51 40 39  0]
 [ 2 16 46 57]
 [57 43 88 47]
 [ 2 72 20 31]
 [30 64 22 18]
 [11 46  6 14]
 [72 86 60 26]
 [65 56 35 12]
 [56 67 90  1]
